In [ ]:
import h2o
from tests import pyunit_utils
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.model.permutation_varimp import permutation_varimp, plot_permutation_var_imp

h2o.init()

Train the model

In [ ]:
prostate_train = h2o.import_file(path=pyunit_utils.locate("smalldata/logreg/prostate_train.csv"))
prostate_train["CAPSULE"] = prostate_train["CAPSULE"].asfactor()

ntrees = 100
learning_rate = 0.1
depth = 5
min_rows = 10
# Build H2O GBM classification model:
gbm_h2o = H2OGradientBoostingEstimator(ntrees=ntrees, learn_rate=learning_rate,
                                       max_depth=depth,
                                       min_rows=min_rows,
                                       distribution="bernoulli")
gbm_h2o.train(x=list(range(1, prostate_train.ncol)), y="CAPSULE", training_frame=prostate_train)

Calculate Permutation Variable Importance

In [ ]:
# select metric
metric = "logloss"

# get pvi df
pm_h2o_df = permutation_varimp(gbm_h2o, prostate_train, use_pandas=True, metric=metric)

# plot pvi
plot_permutation_var_imp(pm_h2o_df, gbm_h2o._model_json["algo"], metric)

